In [ ]:
import os
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import re
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras.layers import MaxPooling3D, \
    Input, ConvLSTM2D, UpSampling2D, \
    MaxPooling2D, Conv2D, Concatenate, Conv3D, \
    Dropout, BatchNormalization, Add, MaxPool2D, Conv2DTranspose, MaxPool3D, Conv3DTranspose, UpSampling3D, \
    concatenate, MaxPooling3D, Bidirectional, TimeDistributed 
from tensorflow.python.client import device_lib
import random
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from keras import backend as K 
import neptune
from keras.callbacks import Callback

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/PC_uint_cell_dispersion_curves/dataset/')
x_train = np.load('train_x.npy')
y_train = np.load('train_y.npy')

x_train = x_train[:258]
y_train =y_train[0:258]

x_val = x_train[258:]
y_val = y_train[258:]

batches = 1
train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batches)
val_set = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batches)

In [ ]:
height = 512
width =  512
img_size = (height, width)
num_classes = 2
batch_size = 1
num_filters = 16
filter_size = 3
dropout_rate = 0.2

In [ ]:
def convolution_op(time_input, n_filters):
    x = Conv2D(filters=n_filters,
                               kernel_size=(3, 3),
                               strides=1, padding='same',
                               activation='relu')(time_input)
    return x

In [ ]:
def normalize_batch(bn_input):
    return keras.layers.BatchNormalization()(bn_input)

In [ ]:
def get_model(img_size, num_classes):
    inputs = Input(shape = (None,None,1))    
    ####################################################################################################################
    c11 = convolution_op(inputs, num_filters)
    BN = normalize_batch(c11)
    c12 = convolution_op(BN, num_filters)
    BN1 = normalize_batch(c12)
    d1 = (MaxPool2D((2, 2), strides=(2, 2)))(BN1)
    d1 = keras.layers.Dropout(dropout_rate)(d1)
    ####################################################################################################################
    c21 = convolution_op(d1, num_filters*2)
    BN = normalize_batch(c21)
    c22 = convolution_op(BN, num_filters*2)
    BN2 = normalize_batch(c22)
    d2 = MaxPool2D((2, 2), strides=(2, 2))(BN2)
    d2 = keras.layers.Dropout(dropout_rate)(d2)
    ####################################################################################################################
    c31 = convolution_op(d2, num_filters*3)
    BN = normalize_batch(c31)
    c32 = convolution_op(BN, num_filters*3)
    BN3 = normalize_batch(c32)
    d3 = MaxPool2D((2, 2), strides=(2, 2))(BN3)
    d3 = keras.layers.Dropout(dropout_rate)(d3)
    ####################################################################################################################
    c41 = convolution_op(d3, num_filters*4)
    BN = normalize_batch(c41)
    c42 = convolution_op(BN, num_filters*4)
    BN4 = normalize_batch(c42)
    d4 = (MaxPool2D((2, 2), strides=(2, 2)))(BN4)
    d4 = keras.layers.Dropout(dropout_rate)(d4)
    ####################################################################################################################
    # bottleneck layer
    ##################################################################################################
    c51 = convolution_op(d4, num_filters*5)
    BN = normalize_batch(c51)
    c52 = convolution_op(BN, num_filters*5)
    BN = normalize_batch(c52)
    ####################################################################################################################
    # Up sampling convolution followed by ConvTranspose
    ####################################################################################################################
    u1 = (UpSampling2D((2, 2)))(BN)
    # skip4 = concatenate([BN4, u1])
    skip4 = keras.layers.Dropout(dropout_rate)(u1)  # adding dropout layer
    c61 = convolution_op(skip4, num_filters*4)
    BN = normalize_batch(c61)
    c62 = convolution_op(BN, num_filters*4)
    BN = normalize_batch(c62)
    ####################################################################################################################
    u2 = (UpSampling2D((2, 1)))(BN)
    # skip3 = concatenate([BN3, u2])
    skip3 = keras.layers.Dropout(dropout_rate)(u2)
    c71 = convolution_op(skip3, num_filters*3)
    BN = normalize_batch(c71)
    c72 = convolution_op(BN, num_filters*3)
    BN = normalize_batch(c72)
    ####################################################################################################################
    u3 = (UpSampling2D((2, 1)))(BN)
    # skip2 = concatenate([BN2, u3])
    skip2 = keras.layers.Dropout(dropout_rate)(u3)
    c81 = convolution_op(skip2, num_filters*2)
    BN = normalize_batch(c81)
    c82 = convolution_op(BN, num_filters*2)
    BN = normalize_batch(c82)
    ####################################################################################################################
    u4 = (UpSampling2D((2, 1)))(BN)
    # skip1 = concatenate([BN1, u4])
    skip1 = keras.layers.Dropout(dropout_rate)(u4)
    c91 = convolution_op(skip1, num_filters)
    BN = normalize_batch(c91)
    c92 = convolution_op(BN, num_filters)
    BN = normalize_batch(c92)
    ####################################################################################################################
    ####################################################################################################################
    # Output layer
    output = Conv2D(1, (1, 1), activation='linear')(BN)
    #################################################################################################################### 
    model_ = Model(inputs=inputs, outputs=output)
    ####################################################################################################################
    return model_

In [ ]:
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = get_model(img_size, num_classes)
model.summary()

In [ ]:
# Configure the model for training.
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
            loss=tf.keras.losses.MSE,
            metrics=[tf.keras.metrics.CosineSimilarity(axis=-1)])


# Train the model, doing validation at the end of each epoch.
class MonitoringCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        for metric_name, metric_value in logs.items():
            neptune.log_metric(metric_name, metric_value)


callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1000, min_delta=0)
model.fit(train_set,
        batch_size=32,
        validation_data=val_set,
        epochs=10000,
        callbacks=[callback])

os.chdir('/home/aijjeh/Desktop/Phd_Projects/PC_uint_cell_dispersion_curves/h5_models/')
model.save("AE_PC_uint_cell.h5")